#Feature Selection and Engineering

## Import of required libraries
Those libraries are used in the notebook to perform the data preparation. Maybee you need to install them first with `pip install <library>`. 

In [ ]:
# Imports
from pandas import json_normalize
import requests
import json
import os
import re
import time
import fnmatch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io import sql

# Download NLTK resources if not already downloaded
nltk.download('punkt')

pd.set_option('display.max_colwidth', 100)

In [1]:
# Read file into dataframe
df_tweets = pd.read_csv('..\\04_Data Cleansing\\04_cleaned_data.csv', sep=',', encoding='utf-8')
print(df_tweets.shape)
df_tweets.head(5)

NameError: name 'pd' is not defined

In [4]:
# Drop the 'tweet_text'
df_tweets.drop(columns=['tweet_text'], inplace=True)

# Rename the 'cleaned' column to 'tweet_text' and length
df_tweets.rename(columns={'cleaned': 'tweet_text'}, inplace=True)
df_tweets.rename(columns={'length': 'char_count'}, inplace=True)

# Display the DataFrame after the changes
df_tweets.head(5)

,interaction_id,tweet_id,senderuser_id,receiveruser_id,cyberbullying_type,interaction_timestamp,char_count,tweet_text
0,1304,1510,876,2009,not_cyberbullying,41:34.9,20,@gyu What is that
1,1998,31503,918,2436,other_cyberbullying,37:48.7,16,@stockputout yes
2,2533,27942,1409,3624,other_cyberbullying,50:46.1,7,
3,2598,52129,1341,3290,none,04:51.4,26,@Blackamazon EXACTLY THIS
4,3511,54626,831,2812,none,11:11.2,34,@Oddtankout http t co KmMJEUME


In [5]:
df_tweets['cyberbullying_type'].unique()

array(['not_cyberbullying', 'other_cyberbullying', 'none', 'gender',
       'sexism', 'age', 'ethnicity', 'religion', 'racism', nan],
      dtype=object)

In [6]:
# Make all the text to uppercase
df_tweets['tweet_text'] = df_tweets['tweet_text'].str.upper()

In [7]:
# ChatGTP has created the pattern 
# Create a pattern which can be used to search the variable 'tweet_text'
patternHarass = '(INSULT)|(ABUSE)|(THREATEN)|(VICTIM)|(TORMENT)|(PERSECUTE)|(INTIMIDATE)|(AGGRESSION)|(RELENTLESS)|(BULLYING)|(HOSTILITY)|(SLANDER)|(HURTFUL)|(ATTACKS)|(OFFENSE)|(MALICIOUS)|(STALKER)|(TAUNT)|(ABUSIVE)|(TARGET)'
patternCyberstalk = '(STALK)|(PURSUE)|(OBSESS)|(WATCH)|(FOLLOW)|(INTRUDE)|(SPY)|(UNWANTED)|(MONITORING)|(SURVEILLANCE)|(TRACKING)|(INVASION)|(SHADOW)|(PRYING)|(PERSISTENCE)|(STALKING)|(SPYING)|(INTRUSIVE)|(PERVASION)|(PURSUIT)'
patternFlaming = '(PROVOCATION)|(INSULTING)|(AGGRESSIVE)|(HOSTILE)|(OFFEND)|(HEATED)|(ARGUMENTATIVE)|(OFFENSIVE)|(CONTENTIOUS)|(DISPUTE)|(ANTAGONISTIC)|(FIERY)|(INCENDIARY)|(HARSH)|(COMBATIVE)|(CLASHING)|(ARGUMENT)|(PROVOCATIVE)|(SPARRING)|(CONTROVERSIAL)'
patternOuting = '(EXPOSURE)|(REVEAL)|(DISCLOSURE)|(BETRAYAL)|(UNMASKING)|(UNVEILING)|(LEAKAGE)|(PUBLICATION)|(BETRAYED)|(UNCOVERING)|(EXPOSED)|(CONFESSION)|(REVEALING)|(LEAKED)|(PRIVACY)|(PRIVATE)|(DATA)|(DETAILS)|(IDENTIFY)|(PUBLISH)'
patternDoxing = '(DOX)|(INFORMATION)|(REVEAL)|(DATA)|(PERSONAL)|(IDENTITY)|(PRIVACY)|(EXPOSED)|(PUBLISH)|(UNMASKING)|(LEAKED)|(PROFILE)|(ADDRESS)|(DETAILS)|(IDENTIFY)|(PUBLISH)|(REVEALING)|(EXPOSE)|(DATA)|(UNVEILING)'
patternExclusion = '(ISOLATE)|(OSTRACIZE)|(EXCLUDE)|(ALIENATION)|(SHUN)|(REJECT)|(IGNORED)|(OUTSIDER)|(SOLITUDE)|(SEGREGATE)|(ABANDONMENT)|(NEGLECTED)|(ISOLATED)|(MARGINALIZED)|(UNWANTED)|(LONELINESS)|(REJECTION)|(ISOLATION)|(NEGLECT)|(NEGLECTING)'
patternImpersonation = '(FAKE)|(IDENTITY)|(DECEPTION)|(FRAUDULENT)|(PRETEND)|(IMITATE)|(COUNTERFEIT)|(MIMIC)|(PHONY)|(FRAUD)|(FORGERY)|(IMPOSTER)|(DECEIVER)|(FAKE PROFILE)|(FALSE IDENTITY)|(IDENTITY THEFT)|(FRAUDULENT ACCOUNT)|(MIMICKING)|(FORGED)|(COUNTERFEITING)'
patternTrolling = '(TROLL)|(PROVOCATION)|(AGITATION)|(DISRUPTION)|(ANNOYING)|(NUISANCE)|(IRRITATION)|(HARASSMENT)|(MISCHIEF)|(TEASING)|(INSTIGATION)|(OFFENSIVE)|(MOCKERY)|(RIDICULE)|(TAUNT)|(DISTURBANCE)|(BAITING)|(ANTAGONISTIC)|(CONTROVERSIAL)|(PRANKS)'
patternHateSpeech = '(HATE)|(BIGOTRY)|(DISCRIMINATION)|(PREJUDICE)|(STEREOTYPING)|(HOMOPHOBIA)|(MISOGYNY)|(RACIAL)|(SLURS)|(XENOPHOBIA)|(BIAS)|(OFFENSIVE)|(DEROGATORY)|(VILE)|(PREJUDICED)|(HATE-FILLED)|(INTOLERANCE)|(RACIST)|(SEXIST)|(HOMOPHOBIC)'
patternThreats = '(THREATEN)|(VIOLENCE)|(DANGER)|(MENACE)|(INTIMIDATE)|(HARM)|(PERIL)|(WARNING)|(COERCION)|(FEAR)|(HARBINGER)|(TERRORIZE)|(MENACING)|(ASSAULT)|(HOSTILITY)|(AGGRESSION)|(THREATENING)|(THREATENED)|(ABUSIVE)|(VIOLENT)'

# Create new variable 'luxurious' as binary dummy (0/1) variable
df_tweets['harass'] = df_tweets['tweet_text'].str.contains(pat = patternHarass).astype(float)
print(df_tweets['harass'].sum())
df_tweets['cyberstalk'] = df_tweets['tweet_text'].str.contains(pat = patternCyberstalk).astype(float)
print(df_tweets['cyberstalk'].sum())
df_tweets['flaming'] = df_tweets['tweet_text'].str.contains(pat = patternFlaming).astype(float)
print(df_tweets['flaming'].sum())
df_tweets['outing'] = df_tweets['tweet_text'].str.contains(pat = patternOuting).astype(float)
print(df_tweets['outing'].sum())
df_tweets['doxing'] = df_tweets['tweet_text'].str.contains(pat = patternDoxing).astype(float)
print(df_tweets['doxing'].sum())
df_tweets['exclusion'] = df_tweets['tweet_text'].str.contains(pat = patternExclusion).astype(float)
print(df_tweets['exclusion'].sum())
df_tweets['impersonation'] = df_tweets['tweet_text'].str.contains(pat = patternImpersonation).astype(float)
print(df_tweets['impersonation'].sum())
df_tweets['trolling'] = df_tweets['tweet_text'].str.contains(pat = patternTrolling).astype(float)
print(df_tweets['trolling'].sum())
df_tweets['hateSpeech'] = df_tweets['tweet_text'].str.contains(pat = patternHateSpeech).astype(float)
print(df_tweets['hateSpeech'].sum())
df_tweets['threats'] = df_tweets['tweet_text'].str.contains(pat = patternThreats).astype(float)
print(df_tweets['threats'].sum())

# Show values
df_tweets[['tweet_text','harass','cyberstalk','flaming','outing', 'doxing', 'exclusion', 'impersonation', 'trolling', 'hateSpeech', 'threats']]

C:\Users\olive\AppData\Local\Temp\ipykernel_21152\2045690740.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['harass'] = df_tweets['tweet_text'].str.contains(pat = patternHarass).astype(float)


3285.0


C:\Users\olive\AppData\Local\Temp\ipykernel_21152\2045690740.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['cyberstalk'] = df_tweets['tweet_text'].str.contains(pat = patternCyberstalk).astype(float)


2288.0


C:\Users\olive\AppData\Local\Temp\ipykernel_21152\2045690740.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['flaming'] = df_tweets['tweet_text'].str.contains(pat = patternFlaming).astype(float)


...

C:\Users\olive\AppData\Local\Temp\ipykernel_21152\2045690740.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['hateSpeech'] = df_tweets['tweet_text'].str.contains(pat = patternHateSpeech).astype(float)


6890.0


C:\Users\olive\AppData\Local\Temp\ipykernel_21152\2045690740.py:33: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['threats'] = df_tweets['tweet_text'].str.contains(pat = patternThreats).astype(float)


1605.0


,tweet_text,harass,cyberstalk,flaming,outing,doxing,exclusion,impersonation,trolling,hateSpeech,threats
0,@GYU WHAT IS THAT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,@STOCKPUTOUT YES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,@BLACKAMAZON EXACTLY THIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,@ODDTANKOUT HTTP T CO KMMJEUME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
64339,ONLY THING THAT HEATS MY HEAD MORE IS WHEN I S...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64340,FOUND OUT THE GIRL WHO SPREAD (TRUE) RUMOURS A...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
64341,NIÑAS REINAS DEL “BULLYING” EN ESCUELAS PRIVA...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64342,@DIANH AS USUAL YOU ARE A MUSLIM LIAR MAKING ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_tweets['word_count'] = df_tweets['tweet_text'].apply(lambda x: len(str(x).split()))
df_tweets[['tweet_text', 'word_count', 'char_count']].head(5)

,tweet_text,word_count,char_count
0,@GYU WHAT IS THAT,4,20
1,@STOCKPUTOUT YES,2,16
2,,0,7
3,@BLACKAMAZON EXACTLY THIS,3,26
4,@ODDTANKOUT HTTP T CO KMMJEUME,5,34


In [ ]:
# This code has been generated by ChatGPT
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Custom functions to calculate text statistics
def average_word_length(tweet_text):
    if isinstance(tweet_text, str) and len(tweet_text) > 0:
        words = word_tokenize(tweet_text)
        return sum(len(word) for word in words) / len(words) if len(words) > 0 else 0
    else:
        return 0

def average_sentence_length(tweet_text):
    if isinstance(tweet_text, str) and len(tweet_text) > 0:
        sentences = sent_tokenize(tweet_text)
        return len(word_tokenize(tweet_text)) / len(sentences) if len(sentences) > 0 else 0
    else:
        return 0

def count_punctuation(tweet_text):
    if isinstance(tweet_text, str):
        return sum(1 for char in tweet_text if char in string.punctuation)
    else:
        return 0

# Apply custom functions to calculate text statistics
df_tweets['avg_word_length'] = df_tweets['tweet_text'].apply(average_word_length)
df_tweets['avg_sentence_length'] = df_tweets['tweet_text'].apply(average_sentence_length)
df_tweets['punctuation_count'] = df_tweets['tweet_text'].apply(count_punctuation)

# Display the DataFrame with text statistics
df_tweets[['tweet_text', 'avg_word_length', 'avg_sentence_length', 'punctuation_count']].head(5)

In [ ]:
zero_values_count = (df_tweets['avg_word_length'] == 0).sum()
print(f'Number of 0 values in avg_word_length: {zero_values_count}')


Check, on how many 0 Values have been created.

In [120]:
print(f'Number of entries with avg_word_length as 0: {(df_tweets["avg_word_length"] == 0).sum()}')
print(f'Number of entries with avg_sentence_length as 0: {(df_tweets["avg_sentence_length"] == 0).sum()}')
print(f'Number of entries with punctuation_count as 0: {(df_tweets["punctuation_count"] == 0).sum()}')


Number of entries with avg_word_length as 0: 11
Number of entries with avg_sentence_length as 0: 11
Number of entries with punctuation_count as 0: 29609


Looking into the ones that have 0 as a value:

In [121]:
print(df_tweets[df_tweets['avg_word_length'] == 0][['tweet_text', 'avg_word_length']])

      tweet_text  avg_word_length
2                             0.0
145                           0.0
212                           0.0
3434                          0.0
7304                          0.0
13327                         0.0
15911                         0.0
33643        NaN              0.0
40379                         0.0
56275                         0.0
63881                         0.0


In [122]:
print(df_tweets[df_tweets['avg_sentence_length'] == 0][['tweet_text', 'avg_sentence_length']])

      tweet_text  avg_sentence_length
2                                 0.0
145                               0.0
212                               0.0
3434                              0.0
7304                              0.0
13327                             0.0
15911                             0.0
33643        NaN                  0.0
40379                             0.0
56275                             0.0
63881                             0.0


In [123]:
print(df_tweets[df_tweets['punctuation_count'] == 0][['tweet_text', 'punctuation_count']])

                                              tweet_text  punctuation_count
2                                                                         0
5                                                   ROFL                  0
6                               OH  YES HE IS A BAD GUY                   0
7                                 HTTP   T CO NXGTQSLCHG                  0
9                                         GAY RAPE JOKE                   0
...                                                  ...                ...
64334  STILL DOESN T CHANGE THE FACT THAT YOU ARE A P...                  0
64336  I AM A WHITE FEMALE  EVERY DAY IN HIGH SCHOOL ...                  0
64338   MKR HEY PETE  MAYBE WHAT THOSE KIDS NEED IS S...                  0
64339  ONLY THING THAT HEATS MY HEAD MORE IS WHEN I S...                  0
64341  NIÑAS  REINAS DEL “BULLYING” EN ESCUELAS PRIVA...                  0

[29609 rows x 2 columns]


The ones that have 0 values are explainable due to the values presented, like the punctuation count is 0 because there is no punctuation.

In [124]:
df_tweets.dtypes

interaction_id             int64
tweet_id                   int64
senderuser_id              int64
receiveruser_id            int64
cyberbullying_type        object
interaction_timestamp     object
char_count                 int64
tweet_text                object
harass                   float64
cyberstalk               float64
flaming                  float64
outing                   float64
doxing                   float64
exclusion                float64
impersonation            float64
trolling                 float64
hateSpeech               float64
threats                  float64
word_count                 int64
avg_word_length          float64
avg_sentence_length      float64
punctuation_count          int64
dtype: object

In [125]:
df_tweets.to_csv('..\\06_Feature Selection & Engineering\\06_dataset_tweets_featured.csv',
                      sep=",",
                      encoding='utf-8',
                      index=False)